In [21]:
%load_ext lab_black
import pandas as pd
from IPython.display import JSON
import warnings

warnings.filterwarnings("ignore")

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [57]:
df = pd.read_csv("world_cup_2022_catar.csv")
df.shape

(64, 25)

In [7]:
df.head(1)

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,...,duels_won_home,duels_won_away,prediction_team_home_win,prediction_draw,prediction_team_away_win,prediction_quantity,location,events_list,lineup_home,lineup_away
0,Final,18/12/2022,True,4,2,Argentina,France,3,3,54%,...,40%,60%,64%,3%,33%,1002332,Lusail Iconic Stadium,"[{'event_team': 'home', 'event_time': "" 23' "",...","[{'player_name': ' Di María ', 'player_number'...","[{'player_name': ' Giroud ', 'player_number': ..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   stage                     64 non-null     object
 1   date                      64 non-null     object
 2   pens                      64 non-null     bool  
 3   pens_home_score           64 non-null     object
 4   pens_away_score           64 non-null     object
 5   team_name_home            64 non-null     object
 6   team_name_away            64 non-null     object
 7   team_home_score           64 non-null     int64 
 8   team_away_score           64 non-null     int64 
 9   possession_home           64 non-null     object
 10  possession_away           64 non-null     object
 11  total_shots_home          64 non-null     int64 
 12  total_shots_away          64 non-null     int64 
 13  shots_on_target_home      64 non-null     int64 
 14  shots_on_target_away      64

In [5]:
df.describe()

,team_home_score,team_away_score,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,prediction_quantity
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,6.400000e+01
mean,1.578125,1.109375,11.296875,11.453125,4.312500,3.765625,4.036118e+05
std,1.551289,1.055856,5.088673,5.866043,2.617857,2.729860,2.001993e+05
min,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.304210e+05
25%,0.000000,0.000000,8.000000,7.750000,3.000000,2.000000,2.321788e+05
50%,1.000000,1.000000,10.500000,10.500000,4.000000,3.000000,3.639255e+05
75%,2.000000,2.000000,14.000000,14.000000,6.000000,5.000000,5.351200e+05
max,7.000000,4.000000,26.000000,32.000000,10.000000,12.000000,1.002332e+06


In [8]:
df.columns

Index(['stage', 'date', 'pens', 'pens_home_score', 'pens_away_score',
       'team_name_home', 'team_name_away', 'team_home_score',
       'team_away_score', 'possession_home', 'possession_away',
       'total_shots_home', 'total_shots_away', 'shots_on_target_home',
       'shots_on_target_away', 'duels_won_home', 'duels_won_away',
       'prediction_team_home_win', 'prediction_draw',
       'prediction_team_away_win', 'prediction_quantity', 'location',
       'events_list', 'lineup_home', 'lineup_away'],
      dtype='object')

### Manipulating events_list, lineup_home and lineup_away columns into new columns

In [65]:
array = [
    {"player_name": " Di María ", "player_number": " 11"},
    {"player_name": " Álvarez ", "player_number": " 9"},
    {"player_name": " Messi ", "player_number": " 10"},
    {"player_name": " Mac Allister ", "player_number": " 20"},
    {"player_name": " Fernández ", "player_number": " 24"},
    {"player_name": " De Paul ", "player_number": " 7"},
    {"player_name": " Tagliafico ", "player_number": " 3"},
    {"player_name": " Otamendi ", "player_number": " 19"},
    {"player_name": " Romero ", "player_number": " 13"},
    {"player_name": " Molina ", "player_number": " 26"},
    {"player_name": " Martínez ", "player_number": " 23"},
]


{'player_name': ' Di María ', 'player_number': ' 11'}


In [58]:
def list_to_dict(row):
    return dict(row)

df['lineup_home'] = df['lineup_home'].apply(list_to_dict)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\caio_barros\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\caio_barros\AppData\Local\Temp\ipykernel_6648\162740870.py", line 4, in <module>
    df['lineup_home'] = df['lineup_home'].apply(list_to_dict)
  File "C:\Users\caio_barros\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py", line 4433, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "C:\Users\caio_barros\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\apply.py", line 1088, in apply
    return self.apply_standard()
  File "C:\Users\caio_barros\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\apply.py", line 1143, in apply_standard
    mapped = lib.map_infer(
  File "pandas\_libs\lib.pyx", line 2870, in pandas._libs.lib.map_infer
  Fil

In [51]:
from pandas.io.json import json_normalize

df = json_normalize(df, 'lineup_home', ['player_name', 'player_number'])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\caio_barros\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\caio_barros\AppData\Local\Temp\ipykernel_6648\1202079821.py", line 3, in <module>
    df = json_normalize(df, 'lineup_home', ['player_name', 'player_number'])
  File "C:\Users\caio_barros\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py", line 59, in wrapper
    return alternative(*args, **kwargs)
  File "C:\Users\caio_barros\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\json\_normalize.py", line 515, in _json_normalize
    _recursive_extract(data, record_path, {}, level=0)
  File "C:\Users\caio_barros\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\json\_normalize.py", line 497, in _recursive_extract
    recs = _pull_records(obj, path[0])
  File "C:\Users\caio_barro